# Analysing Spoofed files in dev set - version 2.0 dataset

## TODO

 * Draw scatter plot of posteriors for our best CNN system to project the distribution of genuine and spoofed features
 * Do this for Development and evaluation set.
 * On Evaluation set you could do this plot seperately for different Replay configurations (total 57 will be too much though)

# Summary of file list for spoofed and genuine files in Development set for analysis

## 1) The top 10 spoofed confident correctly classified files : high scores

                               Gen prob     Spf Prob      Log Liklihood ratio
                             

In [31]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Image

%matplotlib inline

In [32]:
# Load the prediction file
# Predictions file that has three columns: genuin, spoofed prob and log likehood ratio

pred_file='model_3sec_relu_0.5_run8/predictions/dev_prediction.txt'
#new_pred_file = 'model_3sec_relu_0.5_run8/predictions/dev_prediction_with_index.txt'
new_pred_file = 'predictions/dev_prediction_with_index.txt'

In [33]:
%%bash

#cat model_3sec_relu_0.5_run8/predictions/dev_prediction_with_index.txt | head
cat predictions/dev_prediction_with_index.txt | head

0 0.759194 0.240806 1.14826
1 0.833182 0.166818 1.60835
2 0.704341 0.295659 0.868054
3 0.435833 0.564167 -0.25809
4 0.143625 0.856375 -1.7855
5 0.966915 0.0330851 3.37503
6 0.693761 0.306239 0.81776
7 0.946258 0.0537418 2.86832
8 0.952323 0.0476765 2.99447
9 0.9263 0.0737001 2.53119


In [34]:
# Separate out predictions of genuine and spoofed files

all_predictions = []
all_gens = []
all_spoofs = []

with open(new_pred_file) as f:
    all_predictions = [line.strip() for line in f]
    
all_gen_predictions = all_predictions[:760]                    # first 760 files in dev are genuine
all_spf_predictions = all_predictions[760:]                  # files from 761-1710 are spoofed examples

In [35]:
print(len(all_gen_predictions))
print(len(all_spf_predictions))

760
950


In [36]:
# Collect all spoofed files for which spoof class got probability > 90%
# the strongly correctly classified cases
'''
fname='index_file_list/allSpoofIndexList_TP.txt'
count=0
splitIndex = 2                   # 1 for genuine and 2 for spoofed (the third column)
predList = all_spf_predictions   # use all_spf_predictions for spoofed case

#print(len(predList))

with open(fname,'w') as f2:
    for items in predList:
        prob = items.strip().split(' ')[splitIndex]   
        if float(prob) > 0.9:
            #print(items)
            count+=1
            f2.write(items+'\n')            
print(count)
'''

"\nfname='index_file_list/allSpoofIndexList_TP.txt'\ncount=0\nsplitIndex = 2                   # 1 for genuine and 2 for spoofed (the third column)\npredList = all_spf_predictions   # use all_spf_predictions for spoofed case\n\n#print(len(predList))\n\nwith open(fname,'w') as f2:\n    for items in predList:\n        prob = items.strip().split(' ')[splitIndex]   \n        if float(prob) > 0.9:\n            #print(items)\n            count+=1\n            f2.write(items+'\n')            \nprint(count)\n"

In [37]:
%%bash
cat index_file_list/allSpoofIndexList_TP.txt | wc -l

733


## Total examples we use = 733/950 (with spoofed class prob>90%)

***Point***
> We select all those spoofed files that got spoofed probability > 90%. We have 733 spoofed files that we can use SLIME to get a reliable estimate for what the CNN might have learned about the spoofed class. Note that when running slime, for many files the SLIME returns no explanation coz we are only returning positive instance explanation. Thus we remove those files.

> Therefore, these spoofed files have been classified with > 90% probability of being spoofed

## Analysing Time: The top two components from SLIME - True positive Spoofed case

Note, that under time analysis, we have cut our input spectrogram into 10 different temporal components/segments, where each segment correpsonds to

> ***300 mili seconds***

In [49]:
# Show how we cut the spectrogram in timexfrequency

Image("time.png")

In [50]:
# Get temporal explanation from slime

file = 'top_two_explanation_indices/time/spoof_TP_box.txt'  # with signal box for spectrogram

In [51]:
%%bash
cat top_two_explanation_indices/time/spoof_TP_box.txt  | head

0 -999
0 -999
0 -999
0 8
0 -999
0 -999
0 -999
0 1
0 -999
0 -999


In [52]:
def get_top1_top2_list(file):
    with open(file) as f:
        top1 = [int(line.strip().split(' ')[0]) for line in f]
    with open(file) as f:  
        top2 = [int(line.strip().split(' ')[1]) for line in f if int(line.strip().split(' ')[1]) != -999]
    return top1, top2    

In [53]:
# Get top1 and top2 in seperate list
top1, top2 = get_top1_top2_list(file)

In [54]:
#print(top1)

In [55]:
print(len(top1))
print(len(top2))

726
285


In [56]:
def get_counts(datalist, key):
    count=0
    for i in range(0,len(datalist)):
        if datalist[i] == key:
            count+=1
    return count            

In [57]:
def print_component_distribution(comps, predList, order):
    print('Printing component weigting distribution for Top:', order)
    for i in comps:
        print('Component ' + str(i) + ' : ' + str(get_counts(predList, i)))    

In [58]:
comps=[0,1,2,3,4,5,6,7,8,9]    # In time we have 10 segments/components

In [59]:
# Print distribution on top1
order = 1
print_component_distribution(comps, top1, order)

Printing component weigting distribution for Top: 1
Component 0 : 703
Component 1 : 13
Component 2 : 0
Component 3 : 0
Component 4 : 0
Component 5 : 0
Component 6 : 0
Component 7 : 4
Component 8 : 6
Component 9 : 0


### Using the top1 components - given by SLIME (Time segmentation)

> With correct indexing **we got 733/950 spoofed files with 90% probability**. SLIME returned 726 top1 positive explanation. Out of these 726 files we find component 0 to be higly dominating. This strongly confirms our findings:

    Component 0 : 703
    Component 1 : 13
    Component 2 : 0
    Component 3 : 0
    Component 4 : 0
    Component 5 : 0
    Component 6 : 0
    Component 7 : 4
    Component 8 : 6
    Component 9 : 0

> *** Earlier when we were doing mistake (indexing issues) Using Signal box and creating new spectrograms***

    Component 0 : 401
    Component 1 : 10
    Component 2 : 0
    Component 3 : 0
    Component 4 : 0
    Component 5 : 0
    Component 6 : 0
    Component 7 : 1
    Component 8 : 5
    Component 9 : 0

    

*** Observation ***
> Top1 predictions favours the **first 300ms**. We get 703 votes (out of 733) from slime saying that it is looking at first 300 ms for spoofed class.

# Pick 5 audio files for hearing

In [60]:
#random_5_ids = [406,358,924,790,877]   # Spoof True positive, >90% probability
#add 1 to each to access correct file index in real world

#random_5_ids = [407,359,925,791,878]   # Spoof True positive, >90% probability
#base='/homes/bc305/myphd/datasets/ASVSpoof2017_v2.0/ASVspoof2017_V2_dev/'

In [61]:
%%bash
#cp /homes/bc305/myphd/datasets/ASVSpoof2017_v2.0/ASVspoof2017_V2_dev/D_1000878.wav audio_files_hearing/spoof_TP/

#ls audio_files_hearing/spoof_TP/

## Analysing Freq: The top two components from SLIME - True positive Spoofed

Note, that under frequency analysis, we have cut our input spectrogram into 8 different frequency components/segments, where each segment correpsonds to

> ***1000 Hz frequency***

In [62]:
# Show how we cut the spectrogram in timexfrequency

Image("frequency.png")

In [63]:
# Get explanation from SLIME

file = 'top_two_explanation_indices/freq/spoof_TP_box.txt'   # signal box -new specs

In [64]:
%%bash
cat 'top_two_explanation_indices/freq/spoof_TP_box.txt' | head


0 3
3 0
0 3
3 0
0 3
0 3
3 0
3 0
3 0
3 0


In [65]:
# Get top1 and top2 in seperate list
top1, top2 = get_top1_top2_list(file)

In [66]:
print(len(top1))
print(len(top2))

733
725


## Top 1 component distribution - Frequency

In [67]:
comps = [0,1,2,3,4,5,6,7]   # in Frequency we have 8 components

In [68]:
# Print distribution on top1

order = 1
print_component_distribution(comps, top1, order)

Printing component weigting distribution for Top: 1
Component 0 : 85
Component 1 : 3
Component 2 : 0
Component 3 : 528
Component 4 : 0
Component 5 : 86
Component 6 : 31
Component 7 : 0


> With corrected indexing and using proper spoofed files to generate explanation: out of 733 top 90% probability files, 528 files vote for Component 3 that belong to frequency 3000-4000 Hz. 

    Component 0 : 85
    Component 1 : 3
    Component 2 : 0
    Component 3 : 528
    Component 4 : 0
    Component 5 : 86
    Component 6 : 31
    Component 7 : 0


> (with indexing mistake) Using new spectrograms, ***with signal box***

    Component 0 : 158
    Component 1 : 5
    Component 2 : 9
    Component 3 : 415
    Component 4 : 0
    Component 5 : 29
    Component 6 : 15
    Component 7 : 0



In [76]:
# Plot the distribution
#plt.plot(top1,'.')

***Observation***
> We see that under frequency analysis, the top explanation appears to be favouring frequency bin 3000-4000Hz for component 3. This is different from what we saw for the genuine class (that favours highly for component 8, 7000-8000Hz)

***Out of total 733 spoofed files***
 1. 528 belongs to 4000Hz
 

**Observation: **
> In SLIME code, as I was returning two indices, for spoofed files SLIME was only returning positive index, which in many cases was only one, therefore, I put -999 as the second value (just not to break the code and make changes)

 * It give high importance to component 0, 3



# Pick 5 audio files for hearing

In [74]:
#random_5_ids = [891,153,271,390,647]   # Spoof True positive, >90% probability
#add 1 to each to access correct file index in real world

#random_5_ids = [892,154,272,391,648]   # Spoof True positive, >90% probability
#base='/homes/bc305/myphd/datasets/ASVSpoof2017_v2.0/ASVspoof2017_V2_dev/'

In [75]:
%%bash
#cp /homes/bc305/myphd/datasets/ASVSpoof2017_v2.0/ASVspoof2017_V2_dev/D_1000648.wav audio_files_hearing/spoof_TP/

#ls audio_files_hearing/spoof_TP/

# Overall Summary from this study (considering only top1 explanation)

**Time segmentation**

>We segmented spectrogram into 10 super-samples (components), each of 1000ms, and ran SLIME to generate explanations using 5000 samples. We find that among 655 True-positive spoofed files (with probability > 90%), the highest influence is shown for **component 0** that corresponds to **first 300ms**. We get 234 votes from slime.


**Frequency segmentation**
> We segmented the spectrograms into 8 different frequency bins, each corresponding to 1000Hz and ran SLIME. In this case, we see that the top explanation appears to be favouring frequency bin 7000Hz. The next two frequency information that influences are 4000Hz and 1000Hz.


> ***NOTE: In genuine case we got highest vote for 8000Hz bin***


** Correlation between time,frequency **

> Under time analysis we found that the model is highly influenced for first 300ms signal. This relates to the fact that the model seem to exploit information related to DTMF tone that was used for concatenating the original RedDots genuine signal to create spoofed counterparts. SLIME seems to explain this very fact, as we find 234 votes for the spoofed signals with probability greater than 90%. We may still want to double check this fact by just taking the spoofed files with highest probability !

> Next interesting observation we find is by doing frequency analysis, we see that spoofed class is looking at the high frequency

> To write properly !!

 

> We select all those spoofed files that got spoofed probability > 90%. We have 733 spoofed files that we can use SLIME to get a reliable estimate for what the CNN might have learned about the spoofed class. Note that when running slime, for many files the SLIME returns no explanation coz we are only returning positive instance explanation. Thus we remove those files.

> Therefore, these spoofed files have been classified with > 90% probability of being spoofed

## What CNN has exploited about Spoofed signal from the training data ?

We first collect all the spoofed audio files that has been strongly correctly classified with more than 90% probability by the CNN model. We find 733 audio files (out of 950). Next, we run SLIME algorithm on these files and generate temporal and spectral explanations.


On the temporal side we get 726 (out of 733 test files) as SLIME did  not find positive explanation for 7 files. Distribution shown below are for 726 spoofed files classified correctly with > 90% probabiliy.
Distribution of **temporal explanation**  

    Component 0 : 703
    Component 1 : 13
    Component 2 : 0
    Component 3 : 0
    Component 4 : 0
    Component 5 : 0
    Component 6 : 0
    Component 7 : 4
    Component 8 : 6
    Component 9 : 0
    
See, we get strong vote (703 files out of 726) for component 0. First two components    

Distribution of **Spectral explanation**

    Component 0 : 85
    Component 1 : 3
    Component 2 : 0
    Component 3 : 528
    Component 4 : 0
    Component 5 : 86
    Component 6 : 31
    Component 7 : 0

   
  
On average, the temporal explanation distribution show strong emphasis in the inital frames. We find majority of spoofed audio files has speech onset present in first few mili seconds. This strongly attributes to the explanation distribution shown by SLIME in frequency. The highest influence is shown for spectral component representing information between 3-4 kHz. 

> This might also correspond to the way replayed database was collected, where DTMF tone was used as a marker to concatenate series of short audio utterance and replayed under different replay configurations. So, when the replayed utterance was cut back to seperate out these individual segments the DTMF tone was removed while also removing the initial silence frames thus leaving a strong cue between a genuine and a spoofed class that the deep models seem to be cleverly exploiting for decision making between two classes of data. (Have to double check these statements, as there is no reference paper describing how actually data was collected. This was what Tomi had told me long back. Writing it here just for my reference)


> Therefore, what **conclusion** we get from above is that the model is identifying a file as spoofed using speech information in 3-4 kHz frequency in the initial few frames of the audio signal.
 






